Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

REPO_ROOT = "/home/leon/models/NeuroFlame"
pal = sns.color_palette("tab10")
```

Imports
=======

``` ipython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from scipy.stats import binned_statistic
```

``` ipython
import sys
sys.path.insert(0, '../../../')

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl, decode_bump_torch
from src.lr_utils import masked_normalize, clamp_tensor, normalize_tensor
```

``` ipython
import pickle as pkl
import os

def pkl_save(obj, name, path="."):
    os.makedirs(path, exist_ok=True)
    destination = path + "/" + name + ".pkl"
    print("saving to", destination)
    pkl.dump(obj, open(destination, "wb"))


def pkl_load(name, path="."):
    source = path + "/" + name + '.pkl'
    print('loading from', source)
    return pkl.load(open( source, "rb"))

```

Helpers
=======

``` ipython
def map2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return np.where(angles > np.pi, angles - 2 * np.pi, angles)

def map2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return np.where(angles < 0, angles + 2 * np.pi, angles)
```

``` ipython
def maptens2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return torch.where(angles > torch.pi, angles - 2 * torch.pi, angles)

def maptens2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return torch.where(angles < 0, angles + 2 * torch.pi, angles)
```

``` ipython
def add_vlines(model, ax=None):

    if ax is None:
        for i in range(len(model.T_STIM_ON)):
            plt.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)
    else:
        for i in range(len(model.T_STIM_ON)):
            ax.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)

```

Model
=====

``` ipython
kwargs = {
    'GAIN': 1.0,
    'DURATION': 15.0,
    'T_STEADY': 4,

    'T_STIM_ON': [1.0, 5.0, 10.0, 14.0],
    'T_STIM_OFF': [2.0, 6.0, 11.0, 15.0],

    'I0': [0.25, -2.0, 0.25, -2.0],
    'PHI0': [180.0, 180, 180, 180],
    'SIGMA0': [2.0, 0.0, 2.0, 0.0],

    'RANDOM_DELAY': 0,
    'MIN_DELAY': 0,
    'MAX_DELAY': 3,

    'IF_FF_STP': 0,
    'FF_USE': 0.5,
    'TAU_FF_FAC': 0.0,
    'TAU_FF_REC': 0.5,

    'IS_STP': [1, 0, 0, 0],
    'USE': [0.1, 0.03, 0.03, 0.1],
    'TAU_FAC': [2.0, 2.0, 2.0, 0.0],
    'TAU_REC': [0.2, 0.2, 0.2, 0.1],
    'W_STP': [1.0, 3.0, 4.0, 1.0],

    'IF_FF_ADAPT': 0,
    'A_FF_ADAPT': 1.0,
    'TAU_FF_ADAPT': 100.0,

    'IF_ADAPT': 1,
    'A_ADAPT': 1.0,
    'TAU_ADAPT': 100.0,
}
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "train_odr_EI.yml"
DEVICE = 'cuda'
seed = np.random.randint(0, 1e6)

seed = 0
print('seed', seed)
```

``` ipython
N_BATCH = 128*7
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=N_BATCH, **kwargs)
```

``` ipython
model_state_dict = torch.load('../models/odr/odr_%d.pth' % seed)
model.load_state_dict(model_state_dict);
model.eval();
```

``` ipython
print(model.J_STP.item())
# model.J_STP = nn.Parameter(0.98 * model.J_STP)
# print(model.J_STP.item())
```

Simulations
===========

``` ipython
def shifted_phase(phase1, phase2, bias_strength, bias_var, direction=-1):
    """
    shift phase2_original away from phase1 by bias_strength (in radians)
    direction='repulsive' for away, 'attractive' for toward
    All phases in radians
    """
    delta = (phase1 - phase2) * torch.pi / 180.0
    # - for repulsion, + for attraction
    phase2_biased = phase2 + direction * bias_strength * torch.sin(delta + bias_var * torch.randn_like(phase2))  + bias_var * torch.randn_like(phase2)
    return torch.remainder(phase2_biased, 360.0)
```

``` ipython
model.N_BATCH = N_BATCH
model.PHI0 = torch.randint(low=0, high=360, size=(N_BATCH, len(model.I0), 1), device=DEVICE, dtype=torch.float)
model.PHI0_UNBIASED = torch.deg2rad(model.PHI0.clone())

if model.REP_BIAS>0:
   model.PHI0[:, 2] = shifted_phase(model.PHI0[:, 0], model.PHI0[:, 2], model.REP_BIAS, model.REP_VAR)
```

``` ipython
with torch.no_grad():
    ff_input = model.init_ff_input()
    rates_tensor = model.forward(ff_input=ff_input, RET_STP=1)
rates = rates_tensor.cpu().detach().numpy()
print('rates', rates.shape)
```

``` ipython
m0, m1, phi = decode_bump_torch(rates, axis=-1, RET_TENSOR=0)
```

``` ipython
rel_loc = (model.PHI0_UNBIASED[:, 2, 0] - model.PHI0[:, 0, 0]) * 180.0 / torch.pi
rel_loc = (rel_loc + 180) % (360) - 180

error = (model.PHI0_UNBIASED[:, 2, 0] - model.PHI0[:, 2, 0]) * 180 / torch.pi
error = (error + 180) % (360) - 180

# plt.plot(rel_loc.cpu(), error.cpu(), 'o')
# plt.xlabel('Rel. Loc.')
# plt.ylabel('Input Bias (°)')
# plt.show()
```

Dynamics
========

``` ipython
fig, ax = plt.subplots(1, 1, figsize=[2.5*width, height])

idx = np.random.randint(0, model.N_BATCH)
vmin, vmax = np.percentile(rates[idx].reshape(-1), [5, 95])

ax.imshow(rates[idx].T, aspect='auto', cmap='jet', vmin=vmin, vmax=vmax, origin='lower', extent=[0, model.DURATION, 0, model.Na[0].cpu()])
ax.set_ylabel('Pref. Location (°)')
ax.set_yticks(np.linspace(0, model.Na[0].cpu(), 5), np.linspace(0, 360, 5).astype(int))
ax.set_xlabel('Time (s)')
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2.5*width, height])

idx = np.random.randint(0, model.N_BATCH)

xtime = np.linspace(0, model.DURATION, phi.shape[-1])
idx = np.random.randint(0, model.N_BATCH, 8)
ax[1].plot(xtime, m1[idx].T)
ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
ax[1].set_xlabel('Time (s)')
add_vlines(model, ax[1])

ax[2].plot(xtime, phi[idx].T * 180 / np.pi, alpha=0.5)
ax[2].set_yticks(np.linspace(0, 360, 5).astype(int), np.linspace(0, 360, 5).astype(int))
ax[2].set_ylabel('Bump Center (°)')
ax[2].set_xlabel('Time (s)')
add_vlines(model, ax[2])
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

idx = np.random.randint(0, model.N_BATCH, 5)

for i in idx:
    ax[1].plot(xtime, model.x_list.cpu()[i, :, 0])
    ax[0].plot(xtime, model.u_list.cpu()[i, :, 0])

ax[0].set_xlabel('Time (s)')
ax[1].set_xlabel('Time (s)')

add_vlines(model, ax[1])
add_vlines(model, ax[0])

ax[1].set_ylabel('x')
ax[0].set_ylabel('u')
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

m0_x, m1_x, phi_x = decode_bump_torch(model.x_list, axis=-1, RET_TENSOR=0)

idx = np.random.randint(0, model.N_BATCH, 5)

for i in idx:
    ax[0].plot(xtime, m1_x[i])
    ax[1].plot(xtime, phi_x[i])

ax[0].set_xlabel('Time (s)')
ax[1].set_xlabel('Time (s)')

add_vlines(model, ax[0])
add_vlines(model, ax[1])

ax[0].set_ylabel('$\mathcal{F}_1(x)$')
ax[1].set_ylabel('$\\theta_x$')

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

m0_u, m1_u, phi_u = decode_bump_torch(model.u_list, axis=-1, RET_TENSOR=0)

idx = np.random.randint(0, model.N_BATCH, 5)

for i in idx:
    ax[0].plot(xtime, m1_u[i])
    ax[1].plot(xtime, phi_u[i])

ax[0].set_xlabel('Time (s)')
ax[1].set_xlabel('Time (s)')

add_vlines(model, ax[0])
add_vlines(model, ax[1])

ax[0].set_ylabel('$\mathcal{F}_1(u)$')
ax[1].set_ylabel('$\\theta_u$')

plt.show()
```

``` ipython
```

Errors
======

``` ipython
PHI0 = model.PHI0_UNBIASED.cpu().detach().numpy()

target_loc = PHI0[:, 2] * 180 / np.pi

rel_loc = (PHI0[:, 0] - PHI0[:, 2])
rel_loc = (rel_loc + np.pi) % (2 * np.pi) - np.pi
rel_loc *= 180 / np.pi

error_curr = (phi - PHI0[:, 2])
error_curr = (error_curr + np.pi) % (2 * np.pi) - np.pi
error_curr *= 180 / np.pi

error_prev = (phi - PHI0[:, 0])
error_prev = (error_prev + np.pi) % (2 * np.pi) - np.pi
error_prev *= 180 / np.pi

errors = np.stack((error_prev, error_curr))
print(errors.shape, target_loc.shape, rel_loc.shape)
```

``` ipython
time_points = np.linspace(0, model.DURATION, errors.shape[-1])
idx = np.random.randint(errors.shape[1], size=100)

# fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].plot(time_points, errors[0][idx].T, alpha=.4)
# add_vlines(model, ax[0])

# ax[0].set_xlabel('t')
# ax[0].set_ylabel('prev. error (°)')

# ax[1].plot(time_points, errors[1][idx].T, alpha=.4)
# add_vlines(model, ax[1])

ax[1].set_xlabel('t')
ax[1].set_ylabel('curr. error (°)')
plt.show()
```

``` ipython
print(phi.shape, PHI0.shape, model.start_indices.shape, errors.shape)
stim_start = (model.DT * (model.start_indices - model.N_STEADY)).cpu().numpy()
stim_end = (model.DT * (model.end_indices - model.N_STEADY)).cpu().numpy()

stim_start_idx = ((model.start_indices - model.N_STEADY) / model.N_WINDOW - 1).to(int).cpu().numpy()
stim_end_idx = ((model.end_indices - model.N_STEADY) / model.N_WINDOW - 1).to(int).cpu().numpy()

print(stim_start_idx.shape)
```

``` ipython
idx_half = np.array([stim_end_idx[0] + (stim_start_idx[1] - stim_end_idx[0]) / 2.0,stim_end_idx[-2] + (stim_start_idx[-1] - stim_end_idx[-2]) / 2.0], dtype=int)
t_half = np.array([stim_end[0] + (stim_start[1] - stim_end[0]) / 2.0, stim_end[1] + (stim_start[2] - stim_end[1]) / 2.0], dtype=int)
print(t_half+2)
```

``` ipython
end_point = []
for i, j in enumerate([1, 3]):
    end_ = []
    for k in range(errors.shape[1]):
        idx = stim_start_idx[j][k]
        end_.append(errors[i][k][idx])

    end_point.append(end_)

end_point = np.array(end_point)
print(end_point.shape)
```

``` ipython
end_point_half = []
for i, j in enumerate([1, 3]):
    end_ = []
    for k in range(errors.shape[1]):
        idx = idx_half[i][k]
        end_.append(errors[i][k][idx])

    end_point_half.append(end_)

end_point_half = np.array(end_point_half)
print(end_point_half.shape)
```

``` ipython
end_point_zero = []
for i, j in enumerate([0, 2]):
    end_ = []
    for k in range(errors.shape[1]):
        idx = stim_end_idx[j][k]
        end_.append(errors[i][k][idx])

    end_point_zero.append(end_)

end_point_zero = np.array(end_point_zero)
print(end_point_zero.shape)
```

``` ipython
delay_duration = np.array([stim_start[1, 0] - stim_end[0, 0], stim_start[3, 0] - stim_end[2, 0]])

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].hist(end_point[0], bins='auto', color='r', histtype='step', label='%.1f s' % delay_duration[0])
ax[0].hist(end_point_half[0], bins='auto', color='g', histtype='step', label='%.1f s' % (delay_duration[0] / 2))
ax[0].hist(end_point_zero[0], bins='auto', color='b', histtype='step', label='0s')

ax[0].set_xlabel('Prev. Errors (°)')
ax[0].legend(fontsize=12)

ax[1].hist(end_point[1], bins='auto', color='r', histtype='step', label='%.1f s' % delay_duration[1])
ax[1].hist(end_point_half[1], bins='auto', color='g', histtype='step', label='%.1f s' % (delay_duration[1] / 2))
ax[1].hist(end_point_zero[1], bins='auto', color='b', histtype='step', label='0s')

ax[1].set_xlabel('Curr. Errors (°)')
ax[1].legend(fontsize=12)

plt.show()
```

``` ipython
```

Serial Bias
===========

Serial Curves
-------------

``` ipython
def get_correct_error(nbins, df, thresh=25):
    if thresh is not None:
        data = df[(df['errors'] >= -thresh) & (df['errors'] <= thresh)].copy()
    else:
        data = df.copy()

    # 1. Bias-correct both error and error_half
    bin_edges = np.linspace(0, 360, n_bins + 1)
    data['bin_target'] = pd.cut(data['target_loc'], bins=bin_edges, include_lowest=True)
    mean_errors_per_bin = data.groupby('bin_target')['errors'].mean()
    data['adjusted_errors'] = data['errors'] - data['bin_target'].map(mean_errors_per_bin).astype(float)

    # 2. Bin by relative location for both sessions (full version, [-180, 180])
    data['bin_rel'] = pd.cut(data['rel_loc'], bins=n_bins)
    bin_rel = data.groupby('bin_rel')['adjusted_errors'].agg(['mean', 'sem']).reset_index()
    edges = bin_rel['bin_rel'].cat.categories
    centers = (edges.left + edges.right) / 2

    # 3. FLIP SIGN for abs(rel_loc): defects on the left (-) are flipped so all bins reflect the same "direction"
    data['rel_loc_abs'] = np.abs(data['rel_loc'])
    data['bin_rel_abs'] = pd.cut(data['rel_loc_abs'], bins=n_bins, include_lowest=True)

    # Flip errors for abs plot:
    data['adjusted_errors_abs'] = data['adjusted_errors'] * np.sign(data['rel_loc'])

    bin_rel_abs = data.groupby('bin_rel_abs')['adjusted_errors_abs'].agg(['mean', 'sem']).reset_index()
    edges_abs = bin_rel_abs['bin_rel_abs'].cat.categories
    centers_abs = (edges_abs.left + edges_abs.right) / 2

    # 4. Bin by target location for target-centered analysis (optional)
    bin_target = data.groupby('bin_target')['adjusted_errors'].agg(['mean', 'sem']).reset_index()
    edges_target = bin_target['bin_target'].cat.categories
    target_centers = (edges_target.left + edges_target.right) / 2

    return centers, bin_rel, centers_abs, bin_rel_abs
```

``` ipython
n_bins = 16
data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': end_point[1]})
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

# ax[0].plot(data['target_loc'], data['errors'], 'o', alpha=.1)
ax[0].set_xlabel('Target Loc. (°)')
ax[0].set_ylabel('Error (°)')

stt = binned_statistic(data['target_loc'], data['errors'], statistic='mean', bins=n_bins, range=[0, 360])
dstt = np.mean(np.diff(stt.bin_edges))
ax[0].plot(stt.bin_edges[:-1]+dstt/2,stt.statistic,'r')

ax[0].axhline(color='k', linestyle=":")

# ax[1].plot(data['rel_loc'], data['errors'], 'o', alpha=.1)
ax[1].set_xlabel('Rel. Loc. (°)')
ax[1].set_ylabel('Error (°)')

stt = binned_statistic(data['rel_loc'], data['errors'], statistic='mean', bins=n_bins, range=[-180, 180])
dstt = np.mean(np.diff(stt.bin_edges))
ax[1].plot(stt.bin_edges[:-1]+dstt/2, stt.statistic, 'b')

data['rel_loc_abs'] = np.abs(data['rel_loc'])             # Map -180..180 -> 0..180
data['errors_signed'] = data['errors'] * np.sign(data['rel_loc']) # error "toward/away": flip sign for >0

# ax[2].plot(data['rel_loc_abs'], data['errors_signed'], 'o', alpha=0.1)
ax[2].set_xlabel('|Rel. Loc.| (°)')
ax[2].set_ylabel('Error (°)')

bin_stat = binned_statistic(data['rel_loc_abs'], data['errors_signed'], statistic='mean', bins=n_bins, range=[0, 180])
dstt = np.mean(np.diff(bin_stat.bin_edges))
ax[2].plot(bin_stat.bin_edges[:-1] + dstt/2, bin_stat.statistic, 'b')
ax[2].axhline(color='k', linestyle=":")

# plt.savefig('../figures/figs/christos/uncorr_biases.svg', dpi=300)
plt.show()
```

``` ipython
data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': end_point[1]})
centers, bin_rel, centers_abs, bin_rel_abs = get_correct_error(n_bins, data)
```

``` ipython
data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': end_point_half[1]})
centers_half, bin_rel_half, centers_abs_half, bin_rel_abs_half = get_correct_error(n_bins, data)
```

``` ipython
data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': end_point_zero[1]})
centers_zero, bin_rel_zero, centers_abs_zero, bin_rel_abs_zero = get_correct_error(n_bins, data)
```

``` ipython
delay_duration = stim_start[-1] - stim_end[-2]

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers, bin_rel['mean'], 'r', label='full')
ax[0].fill_between(centers, bin_rel['mean'] - bin_rel['sem'], bin_rel['mean'] + bin_rel['sem'], color='r', alpha=0.2)

ax[0].plot(centers, bin_rel_half['mean'], 'g', label='half')
ax[0].fill_between(centers, bin_rel_half['mean'] - bin_rel_half['sem'], bin_rel_half['mean'] + bin_rel_half['sem'], color='g', alpha=0.2)

ax[0].plot(centers, bin_rel_zero['mean'], 'b', label='zero')
ax[0].fill_between(centers, bin_rel_zero['mean'] - bin_rel_zero['sem'], bin_rel_zero['mean'] + bin_rel_zero['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')

ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_abs, bin_rel_abs['mean'], 'r', label='%.1f s' % delay_duration[1])
ax[1].fill_between(centers_abs, bin_rel_abs['mean'] - bin_rel_abs['sem'], bin_rel_abs['mean'] + bin_rel_abs['sem'], color='r', alpha=0.2)

ax[1].plot(centers_abs, bin_rel_abs_half['mean'], 'g', label='%.1f s' % (delay_duration[1] / 2.0))
ax[1].fill_between(centers_abs, bin_rel_abs_half['mean'] - bin_rel_abs_half['sem'], bin_rel_abs_half['mean'] + bin_rel_abs_half['sem'], color='g', alpha=0.2)

ax[1].plot(centers_abs, bin_rel_abs_zero['mean'], 'b', label='0s' )
ax[1].fill_between(centers_abs, bin_rel_abs_zero['mean'] - bin_rel_abs_zero['sem'], bin_rel_abs_zero['mean'] + bin_rel_abs_zero['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Rel. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')

ax[1].legend(fontsize=12, title='Delay', title_fontsize=12)
ax[1].set_xticks(np.linspace(0, 180, 3))

plt.tight_layout()
plt.show()
```

``` ipython
```

Delay Dependency
----------------

``` ipython
delay_point = []
for i in range(errors.shape[1]):
        idx_start = stim_end_idx[2][i]+1
        idx_end = stim_start_idx[3][i]

        end_ = []
        for idx in range(idx_start, idx_end):
                end_.append(errors[1][i][idx])

        delay_point.append(end_)

delay_point = np.array(delay_point)
print(delay_point.shape, errors.shape)
```

``` ipython
import numpy as np
from scipy.optimize import curve_fit

def fit_deriv_gaussian_circular(df, n_bins, target_col='target_loc', error_col='errors', rel_col='rel_loc', n_tries=10, thresh=25):
    if thresh is not None:
        data = df[(df['errors'] >= -thresh) & (df['errors'] <= thresh)].copy()
    else:
        data = df.copy()

    # 1. Compute "adjusted_errors"
    bin_edges = np.linspace(0, 360, n_bins + 1)
    data = data.copy()
    data['bin_target'] = pd.cut(
        data[target_col], bins=bin_edges, include_lowest=True, right=False)
    mean_errors_per_bin = data.groupby('bin_target', observed=False)[error_col].mean()

    data['adjusted_errors'] = (
        data[error_col] - data['bin_target'].map(mean_errors_per_bin).astype(float)
    )

    # 2. Circular binning for kernel fitting
    x = data[rel_col].values
    y = data['adjusted_errors'].values
    bins = np.linspace(-180, 180, n_bins + 1)
    bin_indices = np.digitize(x, bins, right=False) - 1
    bin_indices[bin_indices == n_bins] = 0

    bin_centers = (bins[:-1] + bins[1:]) / 2
    bin_means = np.array([
        y[bin_indices == i].mean() if np.any(bin_indices == i) else np.nan
        for i in range(n_bins)
    ])

    # Guess parameters from the data
    ampl_guess = (np.nanmax(bin_means) - np.nanmin(bin_means)) / 2
    sigma_guess = (np.nanmax(bin_centers) - np.nanmin(bin_centers)) / 4

    # Model
    def deriv_gaussian(x, A, sigma, mu=0):
        return -A * (x - mu) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2)) / (sigma ** 2)

    mask = np.isfinite(bin_means)
    fit_centers = bin_centers[mask]
    fit_means = bin_means[mask]

    best_loss = np.inf
    best_popt = None

    for _ in range(n_tries):
        # Vary around data-driven guess
        p0 = [
            ampl_guess * np.random.uniform(0.0, 10.0),
            sigma_guess * np.random.uniform(1.0, 10.0),
        ]
        try:
            popt, _ = curve_fit(
                deriv_gaussian, fit_centers, fit_means, p0=p0, maxfev=5000)
            residuals = fit_means - deriv_gaussian(fit_centers, *popt)
            loss = np.sum(residuals**2)
            if loss < best_loss:
                best_loss = loss
                best_popt = popt
        except RuntimeError:
            continue

    if best_popt is None:
        raise RuntimeError("Fit did not converge in any of the tries.")

    result = {
        'amplitude_at_90': -best_popt[0] * (90 - 0) * np.exp(-((90 - 0) ** 2) / (2 * best_popt[1] ** 2)) / (best_popt[1] ** 2),
        'bin_centers': bin_centers,
        'bin_means': bin_means,
        'fit': lambda x: deriv_gaussian(x, *best_popt),
        'data': data
    }

    return result

```

``` ipython
from joblib import Parallel, delayed
import numpy as np

def bootstrap_amplitude_at_90(
    data, n_bins, n_boot=100, n_jobs=-1, random_state=None, fit_kwargs=None
):
    # fit_kwargs: dict for extra arguments to fit_deriv_gaussian_circular
    if fit_kwargs is None:
        fit_kwargs = {}
    rng = np.random.RandomState(random_state)

    def _single_boot(random_seed):
        import warnings
        from scipy.optimize import OptimizeWarning
        warnings.simplefilter("ignore", OptimizeWarning)
        np.random.seed(random_seed)
        d_samp = data.sample(frac=1, replace=True, random_state=np.random.randint(0, 2**32))
        try:
            res = fit_deriv_gaussian_circular(d_samp, n_bins, **fit_kwargs)
            return res['amplitude_at_90']
        except Exception:
            return np.nan

    seeds = rng.randint(0, 2**32, size=n_boot)
    results = Parallel(n_jobs=n_jobs)(
        delayed(_single_boot)(s) for s in seeds
    )
    results = np.array([r for r in results if np.isfinite(r)])
    ci = np.percentile(results, [2.5, 97.5])
    return ci
```

``` ipython
n_bins = 16

cmap = plt.get_cmap('Blues')
colors = [cmap((i+1)/ delay_point.shape[1]) for i in range(delay_point.shape[1])]

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

serial_list = []
for i in range(delay_point.shape[1]):
    data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': delay_point[:, i]})

    centers, bin_rel, centers_abs, bin_rel_abs = get_correct_error(n_bins, data)

    ax[0].plot(centers, bin_rel['mean'], color=colors[i])

    idx_max = np.argmax(np.abs(bin_rel['mean'][centers>0]))
    dum = bin_rel['mean'][centers>0]
    serial_max = dum.iloc[idx_max]

    dum = bin_rel['sem'][centers>0]
    serial_std = dum.iloc[idx_max]

    serial_list.append([serial_max, serial_std])

serial_list = np.array(serial_list).T
print(serial_list.shape)
ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].axvline(-60, ls=':', color='k')
ax[0].axvline(60, ls=':', color='k')

delay_duration = stim_start[1, 0] - stim_end[0, 0]
xdelay = np.linspace(0, delay_duration, serial_list.shape[1])

ax[1].plot(xdelay, serial_list[0], '-', label=dum)
ax[1].fill_between(xdelay, serial_list[0] - serial_list[1], serial_list[0] + serial_list[1], color='b', alpha=0.2)
ax[1].set_xlabel('Delay Length (s)')
ax[1].set_ylabel('Serial Bias (°)')

ax[1].axhline(0, ls='--', color='k')
ax[0].axhline(0, ls='--', color = 'k')
# ax[1].legend(fontsize=12)
# plt.savefig('./figures/NIH/10_25/sb_delay_%s.svg' % name)
plt.show()
```

``` ipython
import warnings
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import OptimizeWarning

n_bins = 16

cmap = plt.get_cmap('Blues')
colors = [cmap((i+1)/ delay_point.shape[1]) for i in range(delay_point.shape[1])]

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

serial_list = []
serial_ci = []
for i in range(delay_point.shape[1]):
    data = pd.DataFrame({'target_loc': target_loc[:, -1], 'rel_loc': rel_loc[:, -1], 'errors': delay_point[:, i]})

    result = fit_deriv_gaussian_circular(data, n_bins=n_bins)
    ci = bootstrap_amplitude_at_90(data.copy(), n_bins=n_bins, n_boot=100)

    ax[0].plot(result['bin_centers'], result['fit'](result['bin_centers']), alpha=1, color=colors[i])

    serial_list.append(result['amplitude_at_90'])
    serial_ci.append(ci)

serial_list = np.array(serial_list)
serial_ci = np.array(serial_ci)

ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].axhline(0, ls='--', color='k')
ax[0].axvline(-60, ls=':', color='k')
ax[0].axvline(60, ls=':', color='k')

delay_duration = stim_start[3, 0] - stim_end[2, 0]
xdelay = np.linspace(0, delay_duration, serial_list.shape[0])

ax[1].plot(xdelay, serial_list, '-')
ax[1].fill_between(xdelay, serial_ci[:,0], serial_ci[:,1], color='gray', alpha=0.3, label='95% CI')
ax[1].axhline(0, ls='--', color='k')
ax[1].set_xlabel('Delay Length (s)')
ax[1].set_ylabel('Serial Bias (°)')

plt.show()
```

``` ipython
```